In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
cifar = tf.keras.datasets.cifar10
(X_train, y_train), (X_test, y_test) = cifar.load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [3]:
X_train = X_train/255
X_test = X_test/255

In [4]:
X_train.shape

(50000, 32, 32, 3)

In [5]:
y_train.shape

(50000, 1)

In [6]:
from tensorflow.keras.utils import to_categorical

In [7]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

## Without Transfer Learning

In [8]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout, GlobalAveragePooling2D

In [9]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
model.add(Conv2D(filters=64, kernel_size=5, activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Conv2D(filters=120, kernel_size=5, activation='relu'))
model.add(Flatten())
model.add(Dense(84, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
history = model.fit(X_train, y_train, batch_size=1024, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
49/49 [==============================] - 4s 80ms/step - loss: 1.9305 - accuracy: 0.2953 - val_loss: 1.6915 - val_accuracy: 0.3935
Epoch 2/5
49/49 [==============================] - 3s 65ms/step - loss: 1.5942 - accuracy: 0.4236 - val_loss: 1.4908 - val_accuracy: 0.4602
Epoch 3/5
49/49 [==============================] - 3s 64ms/step - loss: 1.4606 - accuracy: 0.4739 - val_loss: 1.4153 - val_accuracy: 0.4920
Epoch 4/5
49/49 [==============================] - 3s 64ms/step - loss: 1.3689 - accuracy: 0.5107 - val_loss: 1.3271 - val_accuracy: 0.5269
Epoch 5/5
49/49 [==============================] - 3s 65ms/step - loss: 1.2992 - accuracy: 0.5381 - val_loss: 1.2568 - val_accuracy: 0.5524


In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 64)        51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 1, 120)         192120    
_________________________________________________________________
flatten (Flatten)            (None, 120)               0         
_________________________________________________________________
dense (Dense)                (None, 84)                1

## Transfer Learning with VGG16
#### (training entire architecture with trained weights as starting point)

In [12]:
from tensorflow.keras.applications.vgg16 import VGG16

In [13]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32,32,3))

58892288/58889256 [==============================] - 0s 0us/step


In [14]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [15]:
model = Sequential([base_model])
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                32832     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 14,748,170
Trainable params: 14,748,170
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=1024, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
49/49 [==============================] - 46s 937ms/step - loss: 2.2498 - accuracy: 0.1465 - val_loss: 1.9403 - val_accuracy: 0.2214
Epoch 2/5
49/49 [==============================] - 35s 713ms/step - loss: 1.8583 - accuracy: 0.2434 - val_loss: 1.7546 - val_accuracy: 0.2799
Epoch 3/5
49/49 [==============================] - 35s 712ms/step - loss: 1.7191 - accuracy: 0.3193 - val_loss: 1.5661 - val_accuracy: 0.3997
Epoch 4/5
49/49 [==============================] - 35s 715ms/step - loss: 1.4204 - accuracy: 0.4523 - val_loss: 1.5127 - val_accuracy: 0.4518
Epoch 5/5
49/49 [==============================] - 35s 715ms/step - loss: 1.1755 - accuracy: 0.5581 - val_loss: 1.1494 - val_accuracy: 0.5962


## Transfer Learning VGG16
#### Training only top layers (Feature Extraction)


In [18]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32,32,3))

In [19]:
base_model.trainable=False

In [20]:
model = Sequential([base_model])
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [22]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                32832     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                650       
Total params: 14,748,170
Trainable params: 33,482
Non-trainable params: 14,714,688
_________________________________________________________________


In [21]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=1024, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
49/49 [==============================] - 14s 289ms/step - loss: 1.9453 - accuracy: 0.3235 - val_loss: 1.6245 - val_accuracy: 0.4454
Epoch 2/5
49/49 [==============================] - 14s 286ms/step - loss: 1.4936 - accuracy: 0.4924 - val_loss: 1.4266 - val_accuracy: 0.5113
Epoch 3/5
49/49 [==============================] - 14s 286ms/step - loss: 1.3633 - accuracy: 0.5312 - val_loss: 1.3496 - val_accuracy: 0.5346
Epoch 4/5
49/49 [==============================] - 14s 285ms/step - loss: 1.2985 - accuracy: 0.5535 - val_loss: 1.3050 - val_accuracy: 0.5478
Epoch 5/5
49/49 [==============================] - 14s 285ms/step - loss: 1.2580 - accuracy: 0.5682 - val_loss: 1.2753 - val_accuracy: 0.5594


## Transfer Learning (VGG16)
#### Training more layers (Fine Tuning)

In [23]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32,32,3))

In [24]:
for i, layer in enumerate(base_model.layers):
  print(i, layer.name)

0 input_3
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool


In [25]:
for layer in base_model.layers[:16]:
  layer.trainable=False

In [26]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [27]:
model = Sequential([base_model])
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [29]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_3 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                32832     
_________________________________________________________________
dense_7 (Dense)              (None, 10)                650       
Total params: 14,748,170
Trainable params: 4,753,098
Non-trainable params: 9,995,072
_________________________________________________________________


In [28]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=1024, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
49/49 [==============================] - 17s 337ms/step - loss: 1.4539 - accuracy: 0.4857 - val_loss: 1.1120 - val_accuracy: 0.6076
Epoch 2/5
49/49 [==============================] - 16s 333ms/step - loss: 0.9795 - accuracy: 0.6590 - val_loss: 0.9402 - val_accuracy: 0.6675
Epoch 3/5
49/49 [==============================] - 16s 332ms/step - loss: 0.8665 - accuracy: 0.6973 - val_loss: 0.9117 - val_accuracy: 0.6820
Epoch 4/5
49/49 [==============================] - 16s 332ms/step - loss: 0.7748 - accuracy: 0.7266 - val_loss: 0.9096 - val_accuracy: 0.6848
Epoch 5/5
49/49 [==============================] - 16s 332ms/step - loss: 0.7341 - accuracy: 0.7400 - val_loss: 0.8621 - val_accuracy: 0.6977
